# Day 17

In [1]:
from ipynb.fs.defs.utils import read_lines 
puzzle_input = read_lines('day17.txt')

In [2]:
test_input = """2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533""".splitlines()

In [3]:
def sign(n):
    if n > 0:
        return 1
    if n < 0:
        return -1
    return 0

In [4]:
sign(-5)

-1

In [5]:
def get_weight(start_node, end_node, city):
    (x1, y1), _ = start_node
    (x2, y2), _ = end_node
    weight = 0
    while x1 != x2 or y1 != y2:
        x1 += sign(x2 - x1)
        y1 += sign(y2 - y1)
        weight += int(city[y1][x1])
    return weight        

In [6]:
print(get_weight(((0,0), 'S'), ((0, 1), 'A'), test_input))
print(get_weight(((0,0), 'S'), ((0, 2), 'A'), test_input))
print(get_weight(((0,0), 'S'), ((0, 3), 'A'), test_input))
print(get_weight(((2,2), 'S'), ((5, 2), 'L'), test_input))
print(get_weight(((6,2), 'S'), ((3, 2), 'R'), test_input))

3
6
9
11
11


In [7]:
def filter_valid(nodes, city):
    return [((x, y), f) for ((x, y), f) in nodes if x >= 0 and x < len(city[0]) and y >= 0 and y < len(city)]

In [8]:
def get_options(node, last_node, city, min_blocks, max_blocks):
    (x, y), dir = node
    top = [((x, i), 'B') for i in range(y - max_blocks, y + 1 - min_blocks)]
    bot = [((x, i), 'A') for i in range(y + min_blocks, y + max_blocks + 1)]
    left = [((i, y), 'R') for i in range(x - max_blocks, x + 1 - min_blocks)]
    right = [((i, y), 'L') for i in range(x + min_blocks, x + max_blocks + 1)]
    if dir == 'A':
        node_list = [*left, *right]
    elif dir == 'B':
        node_list = [*left, *right]
    elif dir == 'L':
        node_list = [*top, *bot]
    elif dir == 'R':
        node_list = [*top, *bot]
    else:
        node_list = [*top, *bot, *left, *right]
    valid_nodes = filter_valid(node_list, city)
    return [(dest, get_weight(node, dest, city)) for dest in valid_nodes]

In [9]:
get_options(((0,0), 'S'), ((0,0), 'S'), test_input, 1, 3)

[(((0, 1), 'A'), 3),
 (((0, 2), 'A'), 6),
 (((0, 3), 'A'), 9),
 (((1, 0), 'L'), 4),
 (((2, 0), 'L'), 5),
 (((3, 0), 'L'), 8)]

In [10]:
get_options(((2, 0), 'L'), ((0,0), 'S'), test_input, 1, 3)

[(((2, 1), 'A'), 1), (((2, 2), 'A'), 6), (((2, 3), 'A'), 10)]

In [11]:
from collections import defaultdict
import heapq as heap

def dijkstra(startingNode, city, min_blocks = 1, max_blocks = 3 ):
	visited = set()
	parentsMap = {startingNode: None}
	pq = []
	nodeCosts = defaultdict(lambda: float('inf'))
	nodeCosts[startingNode] = 0
	heap.heappush(pq, (0, startingNode))
 
	while pq:
		# go greedily by always extending the shorter cost nodes first
		_, node = heap.heappop(pq)
		visited.add(node)
 
		for adjNode, weight in get_options(node, parentsMap[node], city, min_blocks, max_blocks):
			if adjNode in visited:	continue
				
			newCost = nodeCosts[node] + weight
			if nodeCosts[adjNode] > newCost:
				parentsMap[adjNode] = node
				nodeCosts[adjNode] = newCost
				heap.heappush(pq, (newCost, adjNode))
        
	return parentsMap, nodeCosts


In [12]:
def part1(inp):
    parents, costs = dijkstra(((0,0), 'S'), inp)
    end_nodes = [(((x, y), f), c) for ((x, y), f), c in costs.items() if x == len(inp[0]) - 1 and y == len(inp) - 1]
    print(end_nodes)
    fastest = min(end_nodes, key=lambda x: x[1])[0]
    parent = fastest
    while parent != None:
        print(parent, costs[parent])
        parent = parents[parent]

In [13]:
part1(test_input)

[(((12, 12), 'A'), 104), (((12, 12), 'L'), 102)]
((12, 12), 'L') 102
((11, 12), 'A') 99
((11, 10), 'R') 93
((12, 10), 'A') 87
((12, 7), 'L') 74
((11, 7), 'A') 71
((11, 4), 'L') 55
((10, 4), 'A') 52
((10, 2), 'L') 43
((8, 2), 'A') 37
((8, 0), 'L') 29
((5, 0), 'B') 23
((5, 1), 'L') 20
((2, 1), 'A') 6
((2, 0), 'L') 5
((0, 0), 'S') 0


In [14]:
part1(puzzle_input)

[(((140, 140), 'A'), 668), (((140, 140), 'L'), 668)]
((140, 140), 'A') 668
((140, 137), 'L') 663
((138, 137), 'A') 661
((138, 134), 'L') 657
((136, 134), 'A') 654
((136, 132), 'R') 651
((137, 132), 'A') 649
((137, 129), 'R') 644
((138, 129), 'A') 642
((138, 126), 'R') 639
((139, 126), 'A') 638
((139, 123), 'L') 633
((138, 123), 'A') 632
((138, 120), 'L') 626
((137, 120), 'A') 625
((137, 117), 'L') 619
((136, 117), 'A') 618
((136, 114), 'L') 614
((135, 114), 'A') 612
((135, 111), 'L') 604
((134, 111), 'A') 602
((134, 110), 'L') 599
((133, 110), 'A') 597
((133, 108), 'L') 594
((132, 108), 'A') 593
((132, 106), 'L') 587
((131, 106), 'A') 585
((131, 105), 'L') 583
((130, 105), 'A') 582
((130, 104), 'L') 580
((129, 104), 'A') 579
((129, 101), 'R') 575
((130, 101), 'A') 573
((130, 98), 'L') 564
((129, 98), 'A') 563
((129, 97), 'R') 562
((130, 97), 'A') 561
((130, 95), 'R') 559
((131, 95), 'A') 557
((131, 93), 'R') 555
((132, 93), 'A') 554
((132, 91), 'R') 552
((133, 91), 'A') 549
((133, 88),

In [15]:
def part2(inp, v = False):
    parents, costs = dijkstra(((0,0), 'S'), inp, 4, 10)
    end_nodes = [(((x, y), f), c) for ((x, y), f), c in costs.items() if x == len(inp[0]) - 1 and y == len(inp) - 1]
    print(end_nodes)
    if v:
        fastest = min(end_nodes, key=lambda x: x[1])[0]
        parent = fastest
        while parent != None:
            print(parent, costs[parent])
            parent = parents[parent]

In [16]:
part2(test_input, True)

[(((12, 12), 'A'), 94), (((12, 12), 'L'), 110)]
((12, 12), 'A') 94
((12, 4), 'L') 60
((8, 4), 'A') 39
((8, 0), 'L') 21
((0, 0), 'S') 0


In [17]:
part2(puzzle_input, True)

[(((140, 140), 'L'), 788), (((140, 140), 'A'), 792)]
((140, 140), 'L') 788
((130, 140), 'A') 771
((130, 135), 'L') 762
((121, 135), 'A') 745
((121, 131), 'L') 735
((113, 131), 'B') 717
((113, 135), 'L') 712
((103, 135), 'B') 690
((103, 139), 'L') 684
((96, 139), 'A') 671
((96, 135), 'L') 661
((86, 135), 'B') 635
((86, 140), 'L') 619
((79, 140), 'A') 600
((79, 136), 'L') 591
((71, 136), 'B') 570
((71, 140), 'L') 558
((64, 140), 'A') 545
((64, 136), 'L') 537
((54, 136), 'A') 510
((54, 132), 'L') 498
((44, 132), 'A') 470
((44, 128), 'L') 461
((37, 128), 'A') 449
((37, 124), 'L') 438
((27, 124), 'A') 413
((27, 119), 'L') 401
((23, 119), 'A') 393
((23, 111), 'L') 373
((18, 111), 'A') 361
((18, 107), 'L') 353
((12, 107), 'A') 340
((12, 99), 'L') 322
((8, 99), 'A') 315
((8, 95), 'L') 308
((2, 95), 'A') 290
((2, 86), 'R') 270
((6, 86), 'A') 264
((6, 80), 'L') 250
((1, 80), 'A') 238
((1, 74), 'R') 225
((5, 74), 'A') 217
((5, 64), 'L') 192
((1, 64), 'A') 182
((1, 54), 'R') 157
((6, 54), 'A') 146